# Mongo tutorial

## Prerequisites

### Documentation

You will find all documentation for :
* [Mongo commands](https://docs.mongodb.com/manual/reference/)
* [Mongo python client](http://api.mongodb.com/python/current/api/pymongo/mongo_client.html#pymongo.mongo_client.MongoClient)

### Import libraries

In [2]:
import datetime
from pprint import pprint

import pymongo
from pymongo import MongoClient

In [3]:
client = MongoClient('localhost', 27017)

In [4]:
# let's work in a test_database
db = client.test_database
posts = db.posts

In [5]:
post = {
    "author": "Mike",
    "text": "My first blog post!",
    "tags": ["mongodb", "python", "pymongo"],
    "date": datetime.datetime.utcnow()
}
post_id = posts.insert_one(post).inserted_id
post_id

ObjectId('67d0a0ab41016d04377e7697')

In [6]:
db.list_collection_names()

['posts']

In [7]:
pprint(posts.find_one())

{'_id': ObjectId('67d0a0ab41016d04377e7697'),
 'author': 'Mike',
 'date': datetime.datetime(2025, 3, 11, 20, 44, 27, 678000),
 'tags': ['mongodb', 'python', 'pymongo'],
 'text': 'My first blog post!'}


You can launch a terminal aside, connect to your server with a mongo client and check that the value is present :

```bash
vagrant@nosql:~$ mongo
> show databases;
admin          0.000GB
config         0.000GB
local          0.000GB
test_database  0.000GB
> use test_database;
switched to db test_database
> db.posts.find()
{ 
    "_id" : ObjectId("..."), 
    "author" : "Mike", 
    "text" : "My first blog post!", 
    "tags" : [ "mongodb", "python", "pymongo"], 
    "date" : ISODate("2019-02-10T11:33:47.883Z") 
}
```

## I. Quick start

### First steps

**Q** : Create a document `{msg: 'hello'}` in the `test` collection with `insert_one()`. Fetch it back to display it. What is the `_id` for ?

NB : if the collection doesn't exist yet, MongoDB automatically creates it.

In [11]:
test = db.test
test_id = test.insert_one({"msg": "hello"}).inserted_id

# Fetch and display the document
pprint(test.find_one())

{'_id': ObjectId('67d0a54c41016d04377e7698'), 'msg': 'hello'}


**Q**: Display the number of documents inside the `test` collection

In [12]:
test.count_documents({})

4

### Interacting with a database

We have 2 `.json` files we want to interact with inside the `data` folder. Let's first dump them into a `MovieLens` database, inside `users` and `movies` collections.

For this section, you will need to read a bit on [query operators](https://docs.mongodb.com/manual/reference/operator/query/#query-selectors). Most methods on collections you will use have `filter` as a first parameter, on which you must pass a dictionary of query parameters.

**Q** : In the `MovieLens` database, load `data/movielens_movies.json` into `movies` and `data/movielens_users.json` into `users`. 

Use the dedicated shell command for this : `mongoimport --db <some_db> --collection <some_collection> --file <some_file>` 

In [23]:
import os

# Create MovieLens database
movielens_db = client.MovieLens

# Create collections for movies and users
movies = movielens_db.movies
users = movielens_db.users

# Use mongoimport via os.system to import the JSON files with Docker connection settings

os.system('mongoimport --host localhost --port 27017 --db MovieLens --collection movies --file data/movielens_movies.json')
os.system('mongoimport --host localhost --port 27017 --db MovieLens --collection users --file data/movielens_users.json')

print("MovieLens database created with collections:", movielens_db.list_collection_names())

MovieLens database created with collections: []


sh: mongoimport: command not found
sh: mongoimport: command not found


**Q** : how many users are in the `MovieLens` database ?

In [18]:
users = movielens_db.users
print(f"Number of users in MovieLens: {users.count_documents({})}")

Number of users in MovieLens: 0


**Q** : Display all comedies (the `genres` property equals `Comedy`). 

NB : You will need to find how to go through a `command_cursor`, then use the `pprint` function for a better display of those documents.

**Q** : Fetch and display the `name` and `occupation` for Clifford Johnathan. The second paramater for `find()` ([doc here](https://api.mongodb.com/python/current/api/pymongo/collection.html#pymongo.collection.Collection.find)) is called the `projection` and is used to limit which data to fetch from the query.

**Q**: How many minors (by `age`) have rated movies ?

**Q**: Display science fiction movies ('Sci-Fi') and suspense movies ('Thriller'). This time you need to use a regex to parse genres and look for those values.

**Q**: If we want more advanced textual search, we need a particular index. Use the `create_index()` method to index as [TEXT](https://docs.mongodb.com/manual/core/index-text/) the `genres` field of the `movies` collection.

**Q**: Restart the search for science fiction and thriller movies with the operator `$text`

**Q**: Display the first 30 movies (`limit`) in alphabetical order (`sort`) by title

**Q**: How many users have seen the movie "Star Wars: Episode V - The Empire Strikes Back (1980)" (`_id 1196`) ? The `movies` argument is an array so we should try the [elemMatch](https://docs.mongodb.com/manual/reference/operator/projection/elemMatch/) operator here.

**Q**: And how many gave it a rating of 1 or 2 ?

### Updating data

**Q**: Insert a new user with the properties `name`, `gender` ('M' or'F'), `occupation` and `age`, using the `insert_one()` command. Display it with `find_one()`.

**Q**: Add an appreciation on a viewed movie with `update_one()`, add the movies property containing a table with a document (`movieid`, `rating`, `timestamp` with the value `datetime.datetime.utcnow()`).

You will need to read the documentation on [update operators](https://docs.mongodb.org/manual/reference/operator/update/).

**Q**: Find the number of users who have declared a `programmer` occupation. Modify them so that they are `developer`. Verify your update.

## II. Modelling a blog

We will now model a blog using Mongo. 

First, switch to a new `Blog` database. Each blog post will have the following arguments:

* The author (author field, string type)
* The date (date field, string type in YYYY-MM-DD format)
* The content (field content)
* Tags (field tags, a string array)
* A list of comments (field comments) containing:
 * The author (author field, string type)
 * The date (date field, string type in YYYY-MM-DD format)
 * The content (field content)


**Q**: Create a first post by `rick`, on January 15th, with the tags `mongodb` and `nosql`.

**Q**: Create a second post by `kate`, on January 21, with the tag `nosql` and a comment from `rick` on the same day.

**Q**: Display the author of the last post with the tag `nosql`

**Q**: Add a comment by `jack` on January 25, to `kate`'s post

**Q**: Display all comments by `kate`

## Postquisites

In [ ]:
!mongo test_database --eval 'db.dropDatabase()'

In [ ]:
!mongo MovieLens --eval 'db.dropDatabase()'

In [ ]:
!mongo Blog --eval 'db.dropDatabase()'